In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models import Simple_NN, Trainer_GCN
from datasets.graph_data import graph_from_design, fill_node_x, mask_from_graph
from datasets.class_data import ClassificationData

In [3]:
# Constants
SOURCE = "../raw/dataset_1/"

## Training a simple NN

In [4]:
# Hyperparameters
num_epochs = 250
batch_size = 32

In [5]:
# Data loading from csv
dataset = ClassificationData(SOURCE)
train_dataloader = DataLoader(dataset, batch_size=batch_size)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [7]:
simple_model = Simple_NN(input_size=4, output_size=2).to(device)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
for t in range(num_epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, simple_model, loss_fn, optimizer)
print("Done!")

Done!


## Predicting NN using Trainer GNN

In [11]:
# Preparing data
features = np.genfromtxt(SOURCE + "data_features.csv", delimiter=', ')
targets = np.genfromtxt(SOURCE + "data_targets.csv", delimiter=',')
design = [4, 64, 64, 2]
data = graph_from_design(design)
fill_node_x(data, design, features, targets)
data.input_mask = mask_from_graph(data)

In [12]:
data

Data(design=[4], num_nodes=134, edge_index=[2, 8960], x=[134, 500], input_mask=[134, 1])

In [13]:
# Loading model
trainer_gnn = Trainer_GCN()
trainer_gnn.load_state_dict(torch.load("../model/model"))
trainer_gnn.eval()

Trainer_GCN(
  (conv1): GCNConv(500, 256)
  (conv2): GCNConv(256, 64)
  (denseB): Linear(in_features=64, out_features=1, bias=True)
  (denseW): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
# Process data
out = trainer_gnn(data)

In [15]:
out[1].shape

torch.Size([134, 1])

## Model Comparison

In [16]:
data = dataset[0:10]

In [17]:
input, output = data
input = input.to(device)
output = output.to(device)

In [18]:
out = simple_model(input)
print("Real: Predicted:")
for y, y_pred in zip(out, output):
    print(y.argmax().item(), y_pred.argmax().item())

AttributeError: 'Tensor' object has no attribute 'x'